<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Source-Preparation" data-toc-modified-id="Data-Source-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Source Preparation</a></span></li><li><span><a href="#Headline-News" data-toc-modified-id="Headline-News-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Headline News</a></span><ul class="toc-item"><li><span><a href="#DAX" data-toc-modified-id="DAX-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>DAX</a></span></li><li><span><a href="#Check-RIC's-Validity" data-toc-modified-id="Check-RIC's-Validity-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Check RIC's Validity</a></span></li></ul></li><li><span><a href="#Get-ESG-Score" data-toc-modified-id="Get-ESG-Score-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get ESG Score</a></span></li></ul></div>

# Data Source Preparation

In [29]:
import refinitiv.data.eikon as ek
from IPython.display import HTML
from datetime import datetime, timedelta
import pandas as pd
import time

from credentials import API

ek.set_app_key(API) #ed.ac.uk

In [2]:
FTSE100_df = pd.read_csv('../data/FTSE100.csv').rename(columns={'Query':'RIC'})
SNP500_df = pd.read_csv('../data/SnP500.csv')
DAX_df = pd.read_csv('../data/DAX40.csv')

display(FTSE100_df.head(3))
display(SNP500_df.head(3))
display(DAX_df.head(3))

,No,Ticker,Name,Currency,MarketCap,Price,RIC
0,1,LAND,LAND SECURITIES GROUP PLC ORD 10 2/3P,GBX,"4,682.10",645.4,LAND.L
1,2,BLND,BRITISH LAND CO PLC ORD 25P,GBX,"3,304.12",363.8,BLND.L
2,3,HL.,HARGREAVES LANSDOWN PLC ORD 0.4P,GBX,"3,874.23",831.4,HRGV.L


,#,Company,Symbol,Weight,Price,Chg,% Chg,RIC
0,1,Apple Inc,AAPL,7.409167,171.95,0.11,-0.07%,AAPL.O
1,2,Microsoft Corp,MSFT,6.772895,321.74,7.89,-2.51%,MSFT.O
2,3,Amazon.com Inc,AMZN,3.018000,114.81,-1.94,(-1.66%),AMZN.O


,Company,Ticker,Exchange,Sector,Market Cap,RIC
0,Adidas AG,ADS,Frankfurt Stock Exchange,Personal Goods,€29.3bn,ADSGn.DE
1,Airbus SE,AIR,Euronext Paris,Aerospace & Defence,€99.8bn,AIR.PA
2,Allianz SE,ALV,Frankfurt Stock Exchange,Life Insurance,€85.3bn,ALVG.DE


In [52]:
FTSE_lists = [firm for firm in FTSE100_df['RIC']]
FORTUNE100_lists = [ric for ric in SNP500_df[:100]['RIC']]
DAX_lists = [ric for ric in DAX_df['RIC']]

In [4]:
DAX_df.tail()

,Company,Ticker,Exchange,Sector,Market Cap,RIC
35,Siemens Healthineers AG,SHL,Frankfurt Stock Exchange,Pharmaceuticals & Biotechnology,€60.4bn,SHLG.DE
36,Symrise AG,SY1,Frankfurt Stock Exchange,Personal Goods,€14.8bn,SY1G.DE
37,Volkswagen AG,VOW,Frankfurt Stock Exchange,Automobiles & Parts,€44.6bn,VOWG_p.DE
38,Vonovia SE,VNA,Frankfurt Stock Exchange,Real Estate Investment & Services,€13.9bn,VNAn.DE
39,Zalando SE,ZAL,Frankfurt Stock Exchange,Support Services,€8.1bn,ZALG.DE


# Headline News

## DAX

In [76]:
def func_collect_news_data(lists, interval = 15):
    
    master_df = pd.DataFrame()
    cnt2 = 0
        
    for ticker in lists: 
        
        init_date = datetime(2022, 1, 1).date()
        now_date = datetime.now().date()
        cnt = 0

        reps = ((now_date - init_date) / interval).days

        for rep in range(reps):
            
            to_date = init_date + timedelta(days = interval)

            str_from = datetime.strftime(init_date, '%Y-%m-%d')
            str_to = datetime.strftime(to_date, '%Y-%m-%d')
            
            query = f'( Topic:NEWS1 OR Topic:TOPALL OR NS:RTRS) AND R:{ticker} AND Language:LEN'
            
            temp_df = ek.get_news_headlines( query = query
                                , count=100
                                , date_from = str_from
                                , date_to = str_to 
                    )
            
            cnt+=temp_df.shape[0]
            temp_df['firm'] = ticker
            master_df = pd.concat([master_df, temp_df])

            init_date = datetime.strptime(str_to, '%Y-%m-%d')

            if temp_df.shape[0] != 0:
                print(ticker, str_from, '-' , str_to, ':=', temp_df.shape[0], 'news headline')
                
        cnt2+=1
        print('---'*20, f'{ticker} is done with total rows: {cnt}',  '\n')
        print(cnt2)
        if cnt2%5 == 0:
            time.sleep(60)
            master_df.to_csv('../data/output_news_headline_GIVEN_Update.csv')
            print('Done downloading...')
        
    print(f'total number of rows:', master_df.shape[0])
    return master_df

In [6]:
DAX_news_headline_df = func_collect_news_data(DAX_lists)

ADSGn.DE 2022-04-01 - 2022-04-16 := 7 news headline
ADSGn.DE 2022-04-16 - 2022-05-01 := 6 news headline
ADSGn.DE 2022-05-01 - 2022-05-16 := 59 news headline
ADSGn.DE 2022-05-16 - 2022-05-31 := 8 news headline
ADSGn.DE 2022-05-31 - 2022-06-15 := 5 news headline
ADSGn.DE 2022-06-15 - 2022-06-30 := 8 news headline
ADSGn.DE 2022-06-30 - 2022-07-15 := 12 news headline
ADSGn.DE 2022-07-15 - 2022-07-30 := 33 news headline
ADSGn.DE 2022-07-30 - 2022-08-14 := 19 news headline
ADSGn.DE 2022-08-14 - 2022-08-29 := 12 news headline
ADSGn.DE 2022-08-29 - 2022-09-13 := 7 news headline
ADSGn.DE 2022-09-13 - 2022-09-28 := 12 news headline
ADSGn.DE 2022-09-28 - 2022-10-13 := 23 news headline
ADSGn.DE 2022-10-13 - 2022-10-28 := 84 news headline
ADSGn.DE 2022-10-28 - 2022-11-12 := 62 news headline
ADSGn.DE 2022-11-12 - 2022-11-27 := 23 news headline
ADSGn.DE 2022-11-27 - 2022-12-12 := 20 news headline
ADSGn.DE 2022-12-12 - 2022-12-27 := 14 news headline
ADSGn.DE 2022-12-27 - 2023-01-11 := 6 news headline


BAYGn.DE 2023-06-10 - 2023-06-25 := 35 news headline
------------------------------------------------------------ BAYGn.DE is done with total rows: 950 

BMWG.DE 2022-04-01 - 2022-04-16 := 10 news headline
BMWG.DE 2022-04-16 - 2022-05-01 := 27 news headline
BMWG.DE 2022-05-01 - 2022-05-16 := 55 news headline
BMWG.DE 2022-05-16 - 2022-05-31 := 9 news headline
BMWG.DE 2022-05-31 - 2022-06-15 := 19 news headline
BMWG.DE 2022-06-15 - 2022-06-30 := 24 news headline
BMWG.DE 2022-06-30 - 2022-07-15 := 28 news headline
BMWG.DE 2022-07-15 - 2022-07-30 := 25 news headline
BMWG.DE 2022-07-30 - 2022-08-14 := 55 news headline
BMWG.DE 2022-08-14 - 2022-08-29 := 17 news headline
BMWG.DE 2022-08-29 - 2022-09-13 := 22 news headline
BMWG.DE 2022-09-13 - 2022-09-28 := 24 news headline
BMWG.DE 2022-09-28 - 2022-10-13 := 43 news headline
BMWG.DE 2022-10-13 - 2022-10-28 := 68 news headline
BMWG.DE 2022-10-28 - 2022-11-12 := 49 news headline
BMWG.DE 2022-11-12 - 2022-11-27 := 26 news headline
BMWG.DE 2022-11

DBKGn.DE 2022-04-16 - 2022-05-01 := 79 news headline
DBKGn.DE 2022-05-01 - 2022-05-16 := 32 news headline
DBKGn.DE 2022-05-16 - 2022-05-31 := 34 news headline
DBKGn.DE 2022-05-31 - 2022-06-15 := 66 news headline
DBKGn.DE 2022-06-15 - 2022-06-30 := 33 news headline
DBKGn.DE 2022-06-30 - 2022-07-15 := 37 news headline
DBKGn.DE 2022-07-15 - 2022-07-30 := 82 news headline
DBKGn.DE 2022-07-30 - 2022-08-14 := 35 news headline
DBKGn.DE 2022-08-14 - 2022-08-29 := 30 news headline
DBKGn.DE 2022-08-29 - 2022-09-13 := 77 news headline
DBKGn.DE 2022-09-13 - 2022-09-28 := 62 news headline
DBKGn.DE 2022-09-28 - 2022-10-13 := 49 news headline
DBKGn.DE 2022-10-13 - 2022-10-28 := 88 news headline
DBKGn.DE 2022-10-28 - 2022-11-12 := 90 news headline
DBKGn.DE 2022-11-12 - 2022-11-27 := 51 news headline
DBKGn.DE 2022-11-27 - 2022-12-12 := 46 news headline
DBKGn.DE 2022-12-12 - 2022-12-27 := 20 news headline
DBKGn.DE 2022-12-27 - 2023-01-11 := 21 news headline
DBKGn.DE 2023-01-11 - 2023-01-26 := 40 news he

EONGn.DE 2022-10-13 - 2022-10-28 := 22 news headline
EONGn.DE 2022-10-28 - 2022-11-12 := 39 news headline
EONGn.DE 2022-11-12 - 2022-11-27 := 25 news headline
EONGn.DE 2022-11-27 - 2022-12-12 := 4 news headline
EONGn.DE 2022-12-12 - 2022-12-27 := 7 news headline
EONGn.DE 2022-12-27 - 2023-01-11 := 3 news headline
EONGn.DE 2023-01-11 - 2023-01-26 := 19 news headline
EONGn.DE 2023-01-26 - 2023-02-10 := 27 news headline
EONGn.DE 2023-02-10 - 2023-02-25 := 5 news headline
EONGn.DE 2023-02-25 - 2023-03-12 := 12 news headline
EONGn.DE 2023-03-12 - 2023-03-27 := 83 news headline
EONGn.DE 2023-03-27 - 2023-04-11 := 11 news headline
EONGn.DE 2023-04-11 - 2023-04-26 := 9 news headline
EONGn.DE 2023-04-26 - 2023-05-11 := 12 news headline
EONGn.DE 2023-05-11 - 2023-05-26 := 15 news headline
EONGn.DE 2023-05-26 - 2023-06-10 := 3 news headline
EONGn.DE 2023-06-10 - 2023-06-25 := 17 news headline
------------------------------------------------------------ EONGn.DE is done with total rows: 515 

FMEG

HNKG.DE 2023-05-26 - 2023-06-10 := 3 news headline
HNKG.DE 2023-06-10 - 2023-06-25 := 2 news headline
------------------------------------------------------------ HNKG.DE is done with total rows: 302 

IFXGn.De 2022-04-01 - 2022-04-16 := 5 news headline
IFXGn.De 2022-04-16 - 2022-05-01 := 2 news headline
IFXGn.De 2022-05-01 - 2022-05-16 := 27 news headline
IFXGn.De 2022-05-16 - 2022-05-31 := 2 news headline
IFXGn.De 2022-05-31 - 2022-06-15 := 1 news headline
IFXGn.De 2022-06-15 - 2022-06-30 := 1 news headline
IFXGn.De 2022-06-30 - 2022-07-15 := 12 news headline
IFXGn.De 2022-07-15 - 2022-07-30 := 8 news headline
IFXGn.De 2022-07-30 - 2022-08-14 := 34 news headline
IFXGn.De 2022-08-14 - 2022-08-29 := 11 news headline
IFXGn.De 2022-08-29 - 2022-09-13 := 1 news headline
IFXGn.De 2022-09-13 - 2022-09-28 := 4 news headline
IFXGn.De 2022-09-28 - 2022-10-13 := 14 news headline
IFXGn.De 2022-10-13 - 2022-10-28 := 14 news headline
IFXGn.De 2022-10-28 - 2022-11-12 := 7 news headline
IFXGn.De 202

MUVGn.DE 2023-05-11 - 2023-05-26 := 15 news headline
MUVGn.DE 2023-05-26 - 2023-06-10 := 5 news headline
------------------------------------------------------------ MUVGn.DE is done with total rows: 248 

PSHG_p.DE 2022-04-01 - 2022-04-16 := 14 news headline
PSHG_p.DE 2022-04-16 - 2022-05-01 := 11 news headline
PSHG_p.DE 2022-05-01 - 2022-05-16 := 40 news headline
PSHG_p.DE 2022-05-16 - 2022-05-31 := 19 news headline
PSHG_p.DE 2022-05-31 - 2022-06-15 := 16 news headline
PSHG_p.DE 2022-06-15 - 2022-06-30 := 23 news headline
PSHG_p.DE 2022-06-30 - 2022-07-15 := 12 news headline
PSHG_p.DE 2022-07-15 - 2022-07-30 := 35 news headline
PSHG_p.DE 2022-07-30 - 2022-08-14 := 25 news headline
PSHG_p.DE 2022-08-14 - 2022-08-29 := 25 news headline
PSHG_p.DE 2022-08-29 - 2022-09-13 := 66 news headline
PSHG_p.DE 2022-09-13 - 2022-09-28 := 58 news headline
PSHG_p.DE 2022-09-28 - 2022-10-13 := 88 news headline
PSHG_p.DE 2022-10-13 - 2022-10-28 := 25 news headline
PSHG_p.DE 2022-10-28 - 2022-11-12 := 3

SATG.DE 2022-04-01 - 2022-04-16 := 3 news headline
SATG.DE 2022-04-16 - 2022-05-01 := 21 news headline
SATG.DE 2022-05-16 - 2022-05-31 := 6 news headline
SATG.DE 2022-06-30 - 2022-07-15 := 2 news headline
SATG.DE 2022-07-15 - 2022-07-30 := 24 news headline
SATG.DE 2022-07-30 - 2022-08-14 := 13 news headline
SATG.DE 2022-09-13 - 2022-09-28 := 1 news headline
SATG.DE 2022-09-28 - 2022-10-13 := 2 news headline
SATG.DE 2022-10-13 - 2022-10-28 := 22 news headline
SATG.DE 2022-10-28 - 2022-11-12 := 9 news headline
SATG.DE 2022-11-12 - 2022-11-27 := 7 news headline
SATG.DE 2022-11-27 - 2022-12-12 := 7 news headline
SATG.DE 2022-12-27 - 2023-01-11 := 5 news headline
SATG.DE 2023-01-11 - 2023-01-26 := 3 news headline
SATG.DE 2023-01-26 - 2023-02-10 := 25 news headline
SATG.DE 2023-02-10 - 2023-02-25 := 13 news headline
SATG.DE 2023-02-25 - 2023-03-12 := 1 news headline
SATG.DE 2023-03-27 - 2023-04-11 := 18 news headline
SATG.DE 2023-04-11 - 2023-04-26 := 34 news headline
SATG.DE 2023-04-26 - 20

VOWG_p.DE 2022-10-13 - 2022-10-28 := 100 news headline
VOWG_p.DE 2022-10-28 - 2022-11-12 := 100 news headline
VOWG_p.DE 2022-11-12 - 2022-11-27 := 57 news headline
VOWG_p.DE 2022-11-27 - 2022-12-12 := 89 news headline
VOWG_p.DE 2022-12-12 - 2022-12-27 := 67 news headline
VOWG_p.DE 2022-12-27 - 2023-01-11 := 62 news headline
VOWG_p.DE 2023-01-11 - 2023-01-26 := 76 news headline
VOWG_p.DE 2023-01-26 - 2023-02-10 := 88 news headline
VOWG_p.DE 2023-02-10 - 2023-02-25 := 83 news headline
VOWG_p.DE 2023-02-25 - 2023-03-12 := 94 news headline
VOWG_p.DE 2023-03-12 - 2023-03-27 := 100 news headline
VOWG_p.DE 2023-03-27 - 2023-04-11 := 58 news headline
VOWG_p.DE 2023-04-11 - 2023-04-26 := 100 news headline
VOWG_p.DE 2023-04-26 - 2023-05-11 := 100 news headline
VOWG_p.DE 2023-05-11 - 2023-05-26 := 92 news headline
VOWG_p.DE 2023-05-26 - 2023-06-10 := 38 news headline
VOWG_p.DE 2023-06-10 - 2023-06-25 := 100 news headline
------------------------------------------------------------ VOWG_p.DE is do

In [12]:
FTSE_news_headline_df = func_collect_news_data(FTSE_lists, 10)

LAND.L 2022-04-11 - 2022-04-21 := 3 news headline
LAND.L 2022-05-01 - 2022-05-11 := 1 news headline
LAND.L 2022-05-11 - 2022-05-21 := 23 news headline
LAND.L 2022-05-21 - 2022-05-31 := 2 news headline
LAND.L 2022-05-31 - 2022-06-10 := 2 news headline
LAND.L 2022-06-20 - 2022-06-30 := 4 news headline
LAND.L 2022-06-30 - 2022-07-10 := 3 news headline
LAND.L 2022-07-10 - 2022-07-20 := 5 news headline
LAND.L 2022-07-20 - 2022-07-30 := 11 news headline
LAND.L 2022-07-30 - 2022-08-09 := 4 news headline
LAND.L 2022-09-08 - 2022-09-18 := 4 news headline
LAND.L 2022-09-18 - 2022-09-28 := 19 news headline
LAND.L 2022-09-28 - 2022-10-08 := 2 news headline
LAND.L 2022-10-18 - 2022-10-28 := 4 news headline
LAND.L 2022-10-28 - 2022-11-07 := 2 news headline
LAND.L 2022-11-07 - 2022-11-17 := 11 news headline
LAND.L 2022-11-17 - 2022-11-27 := 2 news headline
LAND.L 2022-11-27 - 2022-12-07 := 2 news headline
LAND.L 2022-12-07 - 2022-12-17 := 5 news headline
LAND.L 2022-12-17 - 2022-12-27 := 1 news headl

BARC.L 2022-04-11 - 2022-04-21 := 16 news headline
BARC.L 2022-04-21 - 2022-05-01 := 71 news headline
BARC.L 2022-05-01 - 2022-05-11 := 28 news headline
BARC.L 2022-05-11 - 2022-05-21 := 36 news headline
BARC.L 2022-05-21 - 2022-05-31 := 37 news headline
BARC.L 2022-05-31 - 2022-06-10 := 19 news headline
BARC.L 2022-06-10 - 2022-06-20 := 48 news headline
BARC.L 2022-06-20 - 2022-06-30 := 35 news headline
BARC.L 2022-06-30 - 2022-07-10 := 29 news headline
BARC.L 2022-07-10 - 2022-07-20 := 29 news headline
BARC.L 2022-07-20 - 2022-07-30 := 79 news headline
BARC.L 2022-07-30 - 2022-08-09 := 25 news headline
BARC.L 2022-08-09 - 2022-08-19 := 23 news headline
BARC.L 2022-08-19 - 2022-08-29 := 12 news headline
BARC.L 2022-08-29 - 2022-09-08 := 40 news headline
BARC.L 2022-09-08 - 2022-09-18 := 34 news headline
BARC.L 2022-09-18 - 2022-09-28 := 32 news headline
BARC.L 2022-09-28 - 2022-10-08 := 59 news headline
BARC.L 2022-10-08 - 2022-10-18 := 29 news headline
BARC.L 2022-10-18 - 2022-10-28 

VOD.L 2022-12-17 - 2022-12-27 := 3 news headline
VOD.L 2022-12-27 - 2023-01-06 := 7 news headline
VOD.L 2023-01-06 - 2023-01-16 := 49 news headline
VOD.L 2023-01-16 - 2023-01-26 := 22 news headline
VOD.L 2023-01-26 - 2023-02-05 := 57 news headline
VOD.L 2023-02-05 - 2023-02-15 := 47 news headline
VOD.L 2023-02-15 - 2023-02-25 := 21 news headline
VOD.L 2023-02-25 - 2023-03-07 := 10 news headline
VOD.L 2023-03-07 - 2023-03-17 := 18 news headline
VOD.L 2023-03-17 - 2023-03-27 := 13 news headline
VOD.L 2023-03-27 - 2023-04-06 := 22 news headline
VOD.L 2023-04-06 - 2023-04-16 := 17 news headline
VOD.L 2023-04-16 - 2023-04-26 := 11 news headline
VOD.L 2023-04-26 - 2023-05-06 := 39 news headline
VOD.L 2023-05-06 - 2023-05-16 := 46 news headline
VOD.L 2023-05-16 - 2023-05-26 := 87 news headline
VOD.L 2023-05-26 - 2023-06-05 := 14 news headline
VOD.L 2023-06-05 - 2023-06-15 := 100 news headline
VOD.L 2023-06-15 - 2023-06-25 := 21 news headline
VOD.L 2023-06-25 - 2023-07-05 := 10 news headline
-

WTB.L 2022-07-10 - 2022-07-20 := 1 news headline
WTB.L 2022-07-20 - 2022-07-30 := 3 news headline
WTB.L 2022-07-30 - 2022-08-09 := 1 news headline
WTB.L 2022-08-19 - 2022-08-29 := 5 news headline
WTB.L 2022-08-29 - 2022-09-08 := 2 news headline
WTB.L 2022-09-08 - 2022-09-18 := 5 news headline
WTB.L 2022-09-18 - 2022-09-28 := 1 news headline
WTB.L 2022-09-28 - 2022-10-08 := 1 news headline
WTB.L 2022-10-08 - 2022-10-18 := 2 news headline
WTB.L 2022-10-18 - 2022-10-28 := 15 news headline
WTB.L 2022-10-28 - 2022-11-07 := 1 news headline
WTB.L 2022-11-07 - 2022-11-17 := 1 news headline
WTB.L 2022-11-17 - 2022-11-27 := 1 news headline
WTB.L 2022-11-27 - 2022-12-07 := 5 news headline
WTB.L 2022-12-07 - 2022-12-17 := 4 news headline
WTB.L 2023-01-06 - 2023-01-16 := 21 news headline
WTB.L 2023-01-16 - 2023-01-26 := 4 news headline
WTB.L 2023-01-26 - 2023-02-05 := 2 news headline
WTB.L 2023-02-15 - 2023-02-25 := 1 news headline
WTB.L 2023-02-25 - 2023-03-07 := 1 news headline
WTB.L 2023-03-07 -

ITRK.L 2022-12-07 - 2022-12-17 := 2 news headline
ITRK.L 2022-12-17 - 2022-12-27 := 3 news headline
ITRK.L 2022-12-27 - 2023-01-06 := 1 news headline
ITRK.L 2023-01-06 - 2023-01-16 := 1 news headline
ITRK.L 2023-01-16 - 2023-01-26 := 3 news headline
ITRK.L 2023-01-26 - 2023-02-05 := 1 news headline
ITRK.L 2023-02-05 - 2023-02-15 := 1 news headline
ITRK.L 2023-02-15 - 2023-02-25 := 3 news headline
ITRK.L 2023-02-25 - 2023-03-07 := 21 news headline
ITRK.L 2023-03-07 - 2023-03-17 := 2 news headline
ITRK.L 2023-03-17 - 2023-03-27 := 5 news headline
ITRK.L 2023-03-27 - 2023-04-06 := 4 news headline
ITRK.L 2023-04-06 - 2023-04-16 := 1 news headline
ITRK.L 2023-04-16 - 2023-04-26 := 6 news headline
ITRK.L 2023-04-26 - 2023-05-06 := 3 news headline
ITRK.L 2023-05-06 - 2023-05-16 := 5 news headline
ITRK.L 2023-05-16 - 2023-05-26 := 10 news headline
ITRK.L 2023-05-26 - 2023-06-05 := 4 news headline
ITRK.L 2023-06-05 - 2023-06-15 := 1 news headline
ITRK.L 2023-06-15 - 2023-06-25 := 3 news headlin

...

BDEV.L 2023-06-15 - 2023-06-25 := 12 news headline
BDEV.L 2023-06-25 - 2023-07-05 := 20 news headline
------------------------------------------------------------ BDEV.L is done with total rows: 383 

GLEN.L 2022-04-11 - 2022-04-21 := 27 news headline
GLEN.L 2022-04-21 - 2022-05-01 := 51 news headline
GLEN.L 2022-05-01 - 2022-05-11 := 28 news headline
GLEN.L 2022-05-11 - 2022-05-21 := 12 news headline
GLEN.L 2022-05-21 - 2022-05-31 := 42 news headline
GLEN.L 2022-05-31 - 2022-06-10 := 45 news headline
GLEN.L 2022-06-10 - 2022-06-20 := 26 news headline
GLEN.L 2022-06-20 - 2022-06-30 := 34 news headline
GLEN.L 2022-06-30 - 2022-07-10 := 33 news headline
GLEN.L 2022-07-10 - 2022-07-20 := 25 news headline
GLEN.L 2022-07-20 - 2022-07-30 := 42 news headline
GLEN.L 2022-07-30 - 2022-08-09 := 35 news headline
GLEN.L 2022-08-09 - 2022-08-19 := 20 news headline
GLEN.L 2022-08-19 - 2022-08-29 := 13 news headline
GLEN.L 2022-08-29 - 2022-09-08 := 14 news headline
GLEN.L 2022-09-08 - 2022-09-18 := 

CTEC.L 2022-08-19 - 2022-08-29 := 4 news headline
CTEC.L 2022-09-08 - 2022-09-18 := 3 news headline
CTEC.L 2022-09-18 - 2022-09-28 := 4 news headline
CTEC.L 2022-10-08 - 2022-10-18 := 2 news headline
CTEC.L 2022-10-18 - 2022-10-28 := 3 news headline
CTEC.L 2022-11-07 - 2022-11-17 := 13 news headline
CTEC.L 2022-11-17 - 2022-11-27 := 9 news headline
CTEC.L 2022-12-07 - 2022-12-17 := 2 news headline
CTEC.L 2023-01-06 - 2023-01-16 := 2 news headline
CTEC.L 2023-01-16 - 2023-01-26 := 2 news headline
CTEC.L 2023-02-15 - 2023-02-25 := 1 news headline
CTEC.L 2023-02-25 - 2023-03-07 := 2 news headline
CTEC.L 2023-03-07 - 2023-03-17 := 18 news headline
CTEC.L 2023-03-27 - 2023-04-06 := 1 news headline
CTEC.L 2023-04-16 - 2023-04-26 := 4 news headline
CTEC.L 2023-05-16 - 2023-05-26 := 10 news headline
CTEC.L 2023-05-26 - 2023-06-05 := 3 news headline
CTEC.L 2023-06-05 - 2023-06-15 := 4 news headline
------------------------------------------------------------ CTEC.L is done with total rows: 127 

RR.L 2023-01-06 - 2023-01-16 := 3 news headline
RR.L 2023-01-16 - 2023-01-26 := 7 news headline
RR.L 2023-01-26 - 2023-02-05 := 14 news headline
RR.L 2023-02-05 - 2023-02-15 := 36 news headline
RR.L 2023-02-15 - 2023-02-25 := 51 news headline
RR.L 2023-02-25 - 2023-03-07 := 16 news headline
RR.L 2023-03-07 - 2023-03-17 := 18 news headline
RR.L 2023-03-17 - 2023-03-27 := 15 news headline
RR.L 2023-03-27 - 2023-04-06 := 13 news headline
RR.L 2023-04-06 - 2023-04-16 := 6 news headline
RR.L 2023-04-16 - 2023-04-26 := 2 news headline
RR.L 2023-04-26 - 2023-05-06 := 3 news headline
RR.L 2023-05-06 - 2023-05-16 := 30 news headline
RR.L 2023-05-16 - 2023-05-26 := 13 news headline
RR.L 2023-05-26 - 2023-06-05 := 16 news headline
RR.L 2023-06-05 - 2023-06-15 := 6 news headline
RR.L 2023-06-15 - 2023-06-25 := 28 news headline
RR.L 2023-06-25 - 2023-07-05 := 9 news headline
------------------------------------------------------------ RR.L is done with total rows: 649 

HLMA.L 2022-04-11 - 2022-04-

BRBY.L 2022-04-11 - 2022-04-21 := 7 news headline
BRBY.L 2022-04-21 - 2022-05-01 := 7 news headline
BRBY.L 2022-05-01 - 2022-05-11 := 1 news headline
BRBY.L 2022-05-11 - 2022-05-21 := 32 news headline
BRBY.L 2022-05-21 - 2022-05-31 := 6 news headline
BRBY.L 2022-05-31 - 2022-06-10 := 6 news headline
BRBY.L 2022-06-10 - 2022-06-20 := 4 news headline
BRBY.L 2022-06-20 - 2022-06-30 := 3 news headline
BRBY.L 2022-06-30 - 2022-07-10 := 10 news headline
BRBY.L 2022-07-10 - 2022-07-20 := 48 news headline
BRBY.L 2022-07-20 - 2022-07-30 := 3 news headline
BRBY.L 2022-07-30 - 2022-08-09 := 9 news headline
BRBY.L 2022-08-09 - 2022-08-19 := 3 news headline
BRBY.L 2022-08-19 - 2022-08-29 := 2 news headline
BRBY.L 2022-08-29 - 2022-09-08 := 5 news headline
BRBY.L 2022-09-08 - 2022-09-18 := 6 news headline
BRBY.L 2022-09-18 - 2022-09-28 := 31 news headline
BRBY.L 2022-09-28 - 2022-10-08 := 29 news headline
BRBY.L 2022-10-08 - 2022-10-18 := 14 news headline
BRBY.L 2022-10-18 - 2022-10-28 := 3 news hea

BMEB.L 2022-11-07 - 2022-11-17 := 28 news headline
BMEB.L 2022-11-17 - 2022-11-27 := 3 news headline
BMEB.L 2022-11-27 - 2022-12-07 := 4 news headline
BMEB.L 2022-12-07 - 2022-12-17 := 1 news headline
BMEB.L 2022-12-27 - 2023-01-06 := 19 news headline
BMEB.L 2023-01-06 - 2023-01-16 := 9 news headline
BMEB.L 2023-01-16 - 2023-01-26 := 4 news headline
BMEB.L 2023-01-26 - 2023-02-05 := 8 news headline
BMEB.L 2023-02-05 - 2023-02-15 := 1 news headline
BMEB.L 2023-02-25 - 2023-03-07 := 5 news headline
BMEB.L 2023-03-17 - 2023-03-27 := 1 news headline
BMEB.L 2023-03-27 - 2023-04-06 := 1 news headline
BMEB.L 2023-04-16 - 2023-04-26 := 1 news headline
BMEB.L 2023-04-26 - 2023-05-06 := 2 news headline
BMEB.L 2023-05-16 - 2023-05-26 := 1 news headline
BMEB.L 2023-05-26 - 2023-06-05 := 28 news headline
BMEB.L 2023-06-05 - 2023-06-15 := 4 news headline
BMEB.L 2023-06-15 - 2023-06-25 := 10 news headline
BMEB.L 2023-06-25 - 2023-07-05 := 15 news headline
---------------------------------------------

In [77]:
FORTUNE_news_headline_df = func_collect_news_data(FORTUNE100_lists, 10)

AAPL.O 2022-04-11 - 2022-04-21 := 78 news headline
AAPL.O 2022-04-21 - 2022-05-01 := 100 news headline
AAPL.O 2022-05-01 - 2022-05-11 := 93 news headline
AAPL.O 2022-05-11 - 2022-05-21 := 100 news headline
AAPL.O 2022-05-21 - 2022-05-31 := 83 news headline
AAPL.O 2022-05-31 - 2022-06-10 := 100 news headline
AAPL.O 2022-06-10 - 2022-06-20 := 100 news headline
AAPL.O 2022-06-20 - 2022-06-30 := 84 news headline
AAPL.O 2022-06-30 - 2022-07-10 := 51 news headline
AAPL.O 2022-07-10 - 2022-07-20 := 87 news headline
AAPL.O 2022-07-20 - 2022-07-30 := 100 news headline
AAPL.O 2022-07-30 - 2022-08-09 := 60 news headline
AAPL.O 2022-08-09 - 2022-08-19 := 59 news headline
AAPL.O 2022-08-19 - 2022-08-29 := 63 news headline
AAPL.O 2022-08-29 - 2022-09-08 := 100 news headline
AAPL.O 2022-09-08 - 2022-09-18 := 88 news headline
AAPL.O 2022-09-18 - 2022-09-28 := 77 news headline
AAPL.O 2022-09-28 - 2022-10-08 := 100 news headline
AAPL.O 2022-10-08 - 2022-10-18 := 82 news headline
AAPL.O 2022-10-18 - 2022

NVDA.O 2022-10-08 - 2022-10-18 := 27 news headline
NVDA.O 2022-10-18 - 2022-10-28 := 18 news headline
NVDA.O 2022-10-28 - 2022-11-07 := 5 news headline
NVDA.O 2022-11-07 - 2022-11-17 := 63 news headline
NVDA.O 2022-11-17 - 2022-11-27 := 28 news headline
NVDA.O 2022-11-27 - 2022-12-07 := 23 news headline
NVDA.O 2022-12-07 - 2022-12-17 := 28 news headline
NVDA.O 2022-12-17 - 2022-12-27 := 10 news headline
NVDA.O 2022-12-27 - 2023-01-06 := 33 news headline
NVDA.O 2023-01-06 - 2023-01-16 := 9 news headline
NVDA.O 2023-01-16 - 2023-01-26 := 14 news headline
NVDA.O 2023-01-26 - 2023-02-05 := 25 news headline
NVDA.O 2023-02-05 - 2023-02-15 := 20 news headline
NVDA.O 2023-02-15 - 2023-02-25 := 100 news headline
NVDA.O 2023-02-25 - 2023-03-07 := 28 news headline
NVDA.O 2023-03-07 - 2023-03-17 := 19 news headline
NVDA.O 2023-03-17 - 2023-03-27 := 100 news headline
NVDA.O 2023-03-27 - 2023-04-06 := 28 news headline
NVDA.O 2023-04-06 - 2023-04-16 := 14 news headline
NVDA.O 2023-04-16 - 2023-04-26 

META.O 2022-05-31 - 2022-06-10 := 97 news headline
META.O 2022-06-10 - 2022-06-20 := 100 news headline
META.O 2022-06-20 - 2022-06-30 := 86 news headline
META.O 2022-06-30 - 2022-07-10 := 71 news headline
META.O 2022-07-10 - 2022-07-20 := 81 news headline
META.O 2022-07-20 - 2022-07-30 := 100 news headline
META.O 2022-07-30 - 2022-08-09 := 86 news headline
META.O 2022-08-09 - 2022-08-19 := 83 news headline
META.O 2022-08-19 - 2022-08-29 := 44 news headline
META.O 2022-08-29 - 2022-09-08 := 68 news headline
META.O 2022-09-08 - 2022-09-18 := 82 news headline
META.O 2022-09-18 - 2022-09-28 := 86 news headline
META.O 2022-09-28 - 2022-10-08 := 99 news headline
META.O 2022-10-08 - 2022-10-18 := 73 news headline
META.O 2022-10-18 - 2022-10-28 := 100 news headline
META.O 2022-10-28 - 2022-11-07 := 88 news headline
META.O 2022-11-07 - 2022-11-17 := 100 news headline
META.O 2022-11-17 - 2022-11-27 := 43 news headline
META.O 2022-11-27 - 2022-12-07 := 90 news headline
META.O 2022-12-07 - 2022-12

JNJ 2022-10-18 - 2022-10-28 := 100 news headline
JNJ 2022-10-28 - 2022-11-07 := 48 news headline
JNJ 2022-11-07 - 2022-11-17 := 28 news headline
JNJ 2022-11-17 - 2022-11-27 := 11 news headline
JNJ 2022-11-27 - 2022-12-07 := 38 news headline
JNJ 2022-12-07 - 2022-12-17 := 45 news headline
JNJ 2022-12-17 - 2022-12-27 := 5 news headline
JNJ 2022-12-27 - 2023-01-06 := 22 news headline
JNJ 2023-01-06 - 2023-01-16 := 24 news headline
JNJ 2023-01-16 - 2023-01-26 := 74 news headline
JNJ 2023-01-26 - 2023-02-05 := 44 news headline
JNJ 2023-02-05 - 2023-02-15 := 33 news headline
JNJ 2023-02-15 - 2023-02-25 := 17 news headline
JNJ 2023-02-25 - 2023-03-07 := 27 news headline
JNJ 2023-03-07 - 2023-03-17 := 39 news headline
JNJ 2023-03-17 - 2023-03-27 := 21 news headline
JNJ 2023-03-27 - 2023-04-06 := 58 news headline
JNJ 2023-04-06 - 2023-04-16 := 43 news headline
JNJ 2023-04-16 - 2023-04-26 := 100 news headline
JNJ 2023-04-26 - 2023-05-06 := 58 news headline
JNJ 2023-05-06 - 2023-05-16 := 16 news 

LLY 2022-05-21 - 2022-05-31 := 24 news headline
LLY 2022-05-31 - 2022-06-10 := 31 news headline
LLY 2022-06-10 - 2022-06-20 := 14 news headline
LLY 2022-06-20 - 2022-06-30 := 30 news headline
LLY 2022-06-30 - 2022-07-10 := 4 news headline
LLY 2022-07-10 - 2022-07-20 := 13 news headline
LLY 2022-07-20 - 2022-07-30 := 9 news headline
LLY 2022-07-30 - 2022-08-09 := 61 news headline
LLY 2022-08-09 - 2022-08-19 := 8 news headline
LLY 2022-08-19 - 2022-08-29 := 12 news headline
LLY 2022-08-29 - 2022-09-08 := 15 news headline
LLY 2022-09-08 - 2022-09-18 := 21 news headline
LLY 2022-09-18 - 2022-09-28 := 28 news headline
LLY 2022-09-28 - 2022-10-08 := 33 news headline
LLY 2022-10-08 - 2022-10-18 := 14 news headline
LLY 2022-10-18 - 2022-10-28 := 14 news headline
LLY 2022-10-28 - 2022-11-07 := 63 news headline
LLY 2022-11-07 - 2022-11-17 := 37 news headline
LLY 2022-11-17 - 2022-11-27 := 12 news headline
LLY 2022-11-27 - 2022-12-07 := 34 news headline
LLY 2022-12-07 - 2022-12-17 := 54 news head

...

TJX 2023-01-26 - 2023-02-05 := 7 news headline
TJX 2023-02-05 - 2023-02-15 := 10 news headline
TJX 2023-02-15 - 2023-02-25 := 58 news headline
TJX 2023-02-25 - 2023-03-07 := 6 news headline
TJX 2023-03-07 - 2023-03-17 := 6 news headline
TJX 2023-03-17 - 2023-03-27 := 3 news headline
TJX 2023-03-27 - 2023-04-06 := 8 news headline
TJX 2023-04-06 - 2023-04-16 := 4 news headline
TJX 2023-04-16 - 2023-04-26 := 3 news headline
TJX 2023-04-26 - 2023-05-06 := 8 news headline
TJX 2023-05-06 - 2023-05-16 := 16 news headline
TJX 2023-05-16 - 2023-05-26 := 57 news headline
TJX 2023-05-26 - 2023-06-05 := 4 news headline
TJX 2023-06-05 - 2023-06-15 := 7 news headline
TJX 2023-06-15 - 2023-06-25 := 6 news headline
TJX 2023-06-25 - 2023-07-05 := 7 news headline
------------------------------------------------------------ TJX is done with total rows: 505 

83
ADP.O 2022-04-11 - 2022-04-21 := 2 news headline
ADP.O 2022-04-21 - 2022-05-01 := 20 news headline
ADP.O 2022-05-01 - 2022-05-11 := 4 news headli

C 2023-06-25 - 2023-07-05 := 76 news headline
------------------------------------------------------------ C is done with total rows: 3302 

87
VRTX.O 2022-04-11 - 2022-04-21 := 8 news headline
VRTX.O 2022-04-21 - 2022-05-01 := 11 news headline
VRTX.O 2022-05-01 - 2022-05-11 := 37 news headline
VRTX.O 2022-05-11 - 2022-05-21 := 5 news headline
VRTX.O 2022-05-21 - 2022-05-31 := 2 news headline
VRTX.O 2022-05-31 - 2022-06-10 := 11 news headline
VRTX.O 2022-06-10 - 2022-06-20 := 5 news headline
VRTX.O 2022-06-30 - 2022-07-10 := 9 news headline
VRTX.O 2022-07-10 - 2022-07-20 := 6 news headline
VRTX.O 2022-07-20 - 2022-07-30 := 15 news headline
VRTX.O 2022-07-30 - 2022-08-09 := 20 news headline
VRTX.O 2022-08-09 - 2022-08-19 := 3 news headline
VRTX.O 2022-08-29 - 2022-09-08 := 1 news headline
VRTX.O 2022-09-18 - 2022-09-28 := 5 news headline
VRTX.O 2022-09-28 - 2022-10-08 := 2 news headline
VRTX.O 2022-10-08 - 2022-10-18 := 7 news headline
VRTX.O 2022-10-18 - 2022-10-28 := 15 news headline


TMUS.O 2023-02-25 - 2023-03-07 := 7 news headline
TMUS.O 2023-03-07 - 2023-03-17 := 24 news headline
TMUS.O 2023-03-17 - 2023-03-27 := 2 news headline
TMUS.O 2023-03-27 - 2023-04-06 := 8 news headline
TMUS.O 2023-04-06 - 2023-04-16 := 2 news headline
TMUS.O 2023-04-16 - 2023-04-26 := 12 news headline
TMUS.O 2023-04-26 - 2023-05-06 := 43 news headline
TMUS.O 2023-05-06 - 2023-05-16 := 19 news headline
TMUS.O 2023-05-16 - 2023-05-26 := 1 news headline
TMUS.O 2023-05-26 - 2023-06-05 := 17 news headline
TMUS.O 2023-06-05 - 2023-06-15 := 14 news headline
TMUS.O 2023-06-15 - 2023-06-25 := 3 news headline
------------------------------------------------------------ TMUS.O is done with total rows: 685 

91
CB 2022-04-11 - 2022-04-21 := 4 news headline
CB 2022-04-21 - 2022-05-01 := 22 news headline
CB 2022-05-01 - 2022-05-11 := 5 news headline
CB 2022-05-11 - 2022-05-21 := 5 news headline
CB 2022-05-21 - 2022-05-31 := 4 news headline
CB 2022-05-31 - 2022-06-10 := 8 news headline
CB 2022-06-10 -

REGN.O 2022-09-28 - 2022-10-08 := 10 news headline
REGN.O 2022-10-08 - 2022-10-18 := 17 news headline
REGN.O 2022-10-18 - 2022-10-28 := 8 news headline
REGN.O 2022-10-28 - 2022-11-07 := 28 news headline
REGN.O 2022-11-07 - 2022-11-17 := 12 news headline
REGN.O 2022-11-17 - 2022-11-27 := 8 news headline
REGN.O 2022-11-27 - 2022-12-07 := 4 news headline
REGN.O 2022-12-07 - 2022-12-17 := 21 news headline
REGN.O 2022-12-17 - 2022-12-27 := 12 news headline
REGN.O 2022-12-27 - 2023-01-06 := 4 news headline
REGN.O 2023-01-06 - 2023-01-16 := 16 news headline
REGN.O 2023-01-16 - 2023-01-26 := 2 news headline
REGN.O 2023-01-26 - 2023-02-05 := 50 news headline
REGN.O 2023-02-05 - 2023-02-15 := 10 news headline
REGN.O 2023-02-15 - 2023-02-25 := 13 news headline
REGN.O 2023-02-25 - 2023-03-07 := 9 news headline
REGN.O 2023-03-07 - 2023-03-17 := 4 news headline
REGN.O 2023-03-17 - 2023-03-27 := 49 news headline
REGN.O 2023-03-27 - 2023-04-06 := 24 news headline
REGN.O 2023-04-06 - 2023-04-16 := 7 ne

LRCX.O 2022-06-30 - 2022-07-10 := 4 news headline
LRCX.O 2022-07-10 - 2022-07-20 := 4 news headline
LRCX.O 2022-07-20 - 2022-07-30 := 19 news headline
LRCX.O 2022-07-30 - 2022-08-09 := 1 news headline
LRCX.O 2022-08-09 - 2022-08-19 := 1 news headline
LRCX.O 2022-08-19 - 2022-08-29 := 3 news headline
LRCX.O 2022-08-29 - 2022-09-08 := 2 news headline
LRCX.O 2022-09-08 - 2022-09-18 := 12 news headline
LRCX.O 2022-09-18 - 2022-09-28 := 2 news headline
LRCX.O 2022-09-28 - 2022-10-08 := 15 news headline
LRCX.O 2022-10-08 - 2022-10-18 := 25 news headline
LRCX.O 2022-10-18 - 2022-10-28 := 29 news headline
LRCX.O 2022-11-07 - 2022-11-17 := 5 news headline
LRCX.O 2022-11-17 - 2022-11-27 := 1 news headline
LRCX.O 2022-11-27 - 2022-12-07 := 2 news headline
LRCX.O 2022-12-07 - 2022-12-17 := 2 news headline
LRCX.O 2022-12-17 - 2022-12-27 := 4 news headline
LRCX.O 2023-01-06 - 2023-01-16 := 6 news headline
LRCX.O 2023-01-16 - 2023-01-26 := 23 news headline
LRCX.O 2023-01-26 - 2023-02-05 := 8 news hea

In [7]:
DAX_news_headline_df.to_csv('../data/output_news_headline_DAX40_Update.csv', index=None)

In [14]:
FTSE_news_headline_df.to_csv('../data/output_news_headline_FTSE100_Update.csv', index=None)